# Mon approche du problème du Titanic

Dans ce projet, on s'attaque au célèbre défi du Titanic de Kaggle. L'objectif est de prédire qui a survécu au naufrage en analysant les données des passagers. On va se questionner sur la meilleure façon d'aborder ce problème : Quelles données sont les plus importantes ? Comment traiter les données manquantes ? Quel modèle choisir ?

In [205]:
import pandas as pd
import numpy as np
import plotly.express as px
import re

In [ ]:
# on affiche les 5 premières lignes de train.csv
trainData = pd.read_csv('data/train.csv')
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# on affiche les 5 premières lignes de test.csv
testData = pd.read_csv('data/test.csv')
testData.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


**Informations sur nos données**

In [208]:
trainData.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Quelques points à observer**

- Sur `891` passagers, seulement `38,38 %` ont survécu au naufrage.

- L'âge médian était de `28` ans.

- Prix du billet médian de `14,45` alors que le tarif max était de `512,33`

On voit qu'il manque certaines valeurs, allons plus loin.

In [209]:
# affichage du nombre de valeurs manquantes par colonne
trainData.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Valeurs manquantes par colonne**

`Age` : 177 valeurs manquantes

`Cabin` : 687 valeurs manquantes

`Embarked` : 2 valeurs manquantes

**Peut-on remplacer les valeurs vides ?**

Nous pourrions remplacer les valeurs `Age` manquantes par la moyenne du profil type. Imaginons quelques-uns de ces groupes :

- `femmes ayant un ticket en première classe`

- `hommes ayant un ticket en troisième classe`

On peut imaginer que le remplacement par la moyenne d'âge de ces groupes serait plus pertinente que la moyenne globale.

Vérifions-le.

In [210]:
# vérifier que c'est plus pertinent que la moyenne globale
age_by_group = trainData.groupby(['Sex', 'Pclass'])['Age'].agg(['mean', 'min', 'max', 'count'])

age_by_group

mean   min   max  count
Sex    Pclass                              
female 1       34.611765  2.00  63.0     85
       2       28.722973  2.00  57.0     74
       3       21.750000  0.75  63.0    102
male   1       41.281386  0.92  80.0    101
       2       30.740707  0.67  70.0     99
       3       26.507589  0.42  74.0    253

On observe que les groupes ont bien des moyennes d'`Age` différentes ce qui laisse à penser que c'est une décision qui amenera plus de précision.

Remplaçons maintenant les données manquantes.

In [211]:
# remplir les ages manquants avec la moyenne du profil type en se basant sur le sexe et la classe

# fonction afin de pouvoir reproduire cette étape plus facilement dans la suite de ce projet
def fill_age(data):
    data_filled = data.copy()

    # fonction a utiliser dans transform, on aurait pu utiliser une lambda
    def fillna_with_mean(x):
        return x.fillna(x.mean())

    # remplacer valeurs vides par moyenne du groupe (basé sur sexe + classe)
    data_filled['Age'] = data_filled.groupby(['Sex', 'Pclass'])['Age'].transform(fillna_with_mean)

    # si il reste des valeurs vides > utiliser la moyenne globale en dernier recours
    data_filled['Age'] = data_filled['Age'].fillna(data_filled['Age'].mean())

    return data_filled

trainData_age_filled = fill_age(trainData)
testData_age_filled = fill_age(testData)

trainData_age_filled.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Nous n'avons plus de valeurs `Age` vides

Il reste, cependant, des valeurs vides dans la colonne `Cabin`. 

On remarque que ces valeurs vides concernent `687` lignes ce qui veut dire qu'il est hors de question de les supprimer, d'autant plus que cela peut être une information à part entière. On peut penser qu'il serait plus intéressant de les marquer comme `Unknown`.

In [212]:
def fill_cabin(data):
    data_filled = data.copy()

    data_filled['Cabin'] = data_filled['Cabin'].fillna('Unknown')

    return data_filled

trainData_age_cabin_filled = fill_cabin(trainData_age_filled)
testData_age_cabin_filled = fill_cabin(testData_age_filled)

trainData_age_cabin_filled.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

On n'a plus aucune valeur vides dans la colonne `Cabin` mais il reste `2` infos manquantes dans la colonne `Embarked`.

In [213]:
trainData_age_cabin_filled['Embarked'].value_counts(dropna=False)

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

Une bonne approche pourrait être de remplir les valeurs manquantes par le port le plus commun en fonction de leur classe.

In [214]:
def fill_embarked(data):
    data_filled = data.copy()

    # loop à travers les lignes où embarked est vide
    for i in data_filled[data_filled['Embarked'].isna()].index:
        # je récupère le pclass de chaque ligne où embarked est vide
        pclass = data_filled.loc[i, 'Pclass']

        # je trouve le port le plus commun pour cette classe
        most_common_port = data_filled[data_filled['Pclass'] == pclass]['Embarked'].mode()[0]
        
        data_filled.loc[i, 'Embarked'] = most_common_port
        print(f'index: {i}, Embarked: {data_filled.loc[i, 'Embarked']} success')
    
    return data_filled

trainData_filled = fill_embarked(trainData_age_cabin_filled)
testData_filled = fill_embarked(testData_age_cabin_filled)

trainData_filled.isna().sum()

index: 61, Embarked: S success
index: 829, Embarked: S success


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [215]:
women = trainData.loc[trainData.Sex == 'female']['Survived']
rateWomen = sum(women)/len(women) * 100 # somme de tous les 1 (femmes survivantes) / nombre total de femme 1 ou 0

print(f'Pourcentage de femmes ayant survécu :  {rateWomen:.2f} %') # arrondi à 2 décimales

Pourcentage de femmes ayant survécu :  74.20 %


In [216]:
men = trainData.loc[trainData.Sex == 'male']['Survived']
rateMen = sum(men)/len(men) * 100

print(f'Pourcentage d\'hommes ayant survécu :  {rateMen:.2f} %')

Pourcentage d'hommes ayant survécu :  18.89 %


In [217]:
# création df avec les pourcentages
sex_survival_rates = pd.DataFrame({
    'Sex': ['female', 'male'],
    'Survived_Rate': [rateWomen, rateMen]
})

fig_sex_survived = px.bar(sex_survival_rates,
x='Sex',
y='Survived_Rate',
color='Sex',

title='Taux de survie par sexe',
labels={
'Survived_Rate': 'Taux de survie (%)',
'Sex': 'Sexe',
}
)

fig_sex_survived.show()

In [218]:
age_survival_rate = trainData_filled.groupby('Age')['Survived'].mean() * 100

age_survival_fig = px.line(
    x=age_survival_rate.index,
    y=age_survival_rate.values,

    title='Taux de survie par âge',
    labels={
        'x': 'Âge',
        'y': 'Taux de survie (%)'
    }
)

age_survival_fig.show()

In [ ]:
# ajout de features
fullData = [trainData_filled, testData_filled]

# family size
for i in fullData:
    i['FamilySize'] = i['SibSp'] + i['Parch'] + 1

# is alone
for i in fullData:
    i['IsAlone'] = 0

    # si la condition est true on sélectionne 'IsAlone'
    i.loc[i['FamilySize'] == 1, 'IsAlone'] = 1

# has cabin
for i in fullData:
    i['HasCabin'] = 0

    i.loc[i['Cabin'] != 'Unknown', 'HasCabin'] = 1

# title from name
def get_title(name):
    # on cherche un espace, on capture la chaine de plusieurs lettres si elle est suivie d'un point (échappé avec le \)
    search_result = re.search(r' ([A-Za-z]+)\.', name) # raw string pour que python ne traite pas l'échappement avant regex

    if search_result:
        # group(0): return ' Mrs.' (TOUT le pattern)
        # group(1): return 'Mrs' (le groupe de capture entre parenthèses)
        # group(2): (le 2eme groupe de capture entre parenthèses s'il existe)
        return search_result.group(1)
    return ''

for i in fullData:
    i['Title'] = i['Name'].apply(get_title)

# trainData_filled.head(50)
print(trainData_filled['Title'].value_counts(dropna=False))
print(testData_filled['Title'].value_counts(dropna=False))

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Ms            1
Mme           1
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64
Title
Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dr          1
Dona        1
Name: count, dtype: int64


In [220]:
# récupérer dynamiquement tous les titles rares pour les unifier dans une même catégorie
for i in fullData:
    title_count = i['Title'].value_counts()

    i['Title'] = i['Title'].apply(
        # appliquer le vrai titre s'il en existe plus de 10 dans nos données autrement on applique 'Rare'
        lambda real_title: real_title if title_count[real_title] > 10 else 'Rare' 
        )

print(trainData_filled['Title'].value_counts(dropna=False))
print(testData_filled['Title'].value_counts(dropna=False))

Title
Mr        517
Miss      182
Mrs       125
Master     40
Rare       27
Name: count, dtype: int64
Title
Mr        240
Miss       78
Mrs        72
Master     21
Rare        7
Name: count, dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# variable cible > on va prédire qui a survécu 1 ou n'a pas survécu 0
y = trainData_filled['Survived'] 

# on définit quelles caractéristiques utiliser pour la prédiction
features = [
    'Pclass',
    'Sex',
    'SibSp',
    'Parch',
    'FamilySize',
    'IsAlone',
    'HasCabin',
    'Title',
]

X = pd.get_dummies(trainData_filled[features])
Xtest = pd.get_dummies(testData_filled[features])

model = RandomForestClassifier(
    n_estimators=100, # nombre d'arbres de la forêt > + = plus précis mais plus lent
    max_depth=8, # profondeur max par arbre > évite l'overfitting (quand le modèle s'adapte trop aux données d'entrainement)
    random_state=1 # comme une seed > pour pas que cela ne soit random à chaque fois
)

# entrainement du modèle 
model.fit(X, y)

# mise en application
predictions = model.predict(Xtest)

output = pd.DataFrame({
    'PassengerId': testData_filled.PassengerId,
    'Survived': predictions # les prédictions 0 ou 1
})

# output.to_csv('submission3.csv', index=False)



print('success')


success


In [223]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# diviser les données 80% pour apprendre 20% pour tester l'apprentissage
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y,
    test_size=0.2,
    random_state=1
)

print(f"Groupe d'entraînement : {len(X_train)} passagers")
print(f"Groupe de validation  : {len(X_validation)} passagers")

# voir si les proportions sont similaires
# taux_train = y_train.mean()
# taux_validation = y_validation.mean()

# print(f"entraînement: {taux_train:.1%}")
# print(f"Validation: {taux_validation:.1%}")

# normaliser les données
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
Xtest_scaled = scaler.transform(Xtest)

sgd_model = SGDClassifier(
    loss='log_loss',
    max_iter=1000,
    random_state=1
)

sgd_model.fit(X_train_scaled, y_train)

predictions_validation = sgd_model.predict(X_validation_scaled)

precision = accuracy_score(y_validation, predictions_validation)

print(f"Précision sur la validation : {precision:.5%}")


Groupe d'entraînement : 712 passagers
Groupe de validation  : 179 passagers
Précision sur la validation : 75.41899%
